In [ ]:
!pip install dlib
!pip install imutils

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
import numpy as np
import os
import imutils
import dlib
import cv2

import imageio
from imutils import face_utils

In [ ]:
from google.colab import drive
import os
import pandas as pd

drive.mount('/content/drive',force_remount=True)


dataset_path = '/content/drive/My Drive/dsproj/ads_proj_data/ads_proj_data/dataset/dataset'

if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"The specified path '{dataset_path}' does not exist.")

files = os.listdir(dataset_path)
print(f"Files in the dataset folder: {files}")


In [ ]:

os.listdir('/content/drive/My Drive/dsproj/ads_proj_data/ads_proj_data/dataset/dataset')
predictor = dlib.shape_predictor('/content/drive/My Drive/dsproj/ads_proj_data/ads_proj_data/shape_predictor_68_face_landmarks.dat')

In [ ]:
people = ['F01','F02','F04','F05','F06','F07','F08','F09', 'F10','F11','M01','M02','M04','M07','M08']
#people = ['F01']
data_types = ['words']
folder = ['01','02','03','04','05','06','07','08', '09', '10']
instances = ['01','02','03','04','05','06','07','08', '09', '10']

words = ['Begin', 'Choose', 'Connection', 'Navigation', 'Next', 'Previous', 'Start', 'Stop', 'Hello', 'Web']
words_di = {i:words[i] for i in range(len(words))}

In [ ]:
from skimage.transform import resize
import time

In [ ]:
import numpy as np
import os
import imageio
from skimage.transform import resize
import time


MAX_WIDTH = 100
MAX_HEIGHT = 100
max_seq_length = 22


darkness_factor = 0.7

t1 = time.time()

UNSEEN_VALIDATION_SPLIT = []
UNSEEN_TEST_SPLIT = []

directory = '/content/drive/My Drive/dsproj/ads_proj_data/ads_proj_data/dataset/dataset/'

# Initialize empty lists for data and labels
X_train, y_train = [], []
X_val, y_val = [], []
X_test, y_test = [], []

def simulate_low_light(image, factor=0.7):
    # Reduce pixel values to simulate low-light conditions
    dark_image = factor * image
    dark_image = np.clip(dark_image, 0, 255).astype(np.uint8)
    return dark_image

def random_flip(image):
    # Randomly flip the image horizontally
    if np.random.rand() < 0.5:
        return np.fliplr(image)
    return image

def pad_or_truncate(sequence):
    if len(sequence) < max_seq_length:
        pad_array = np.zeros((max_seq_length - len(sequence), MAX_WIDTH, MAX_HEIGHT, 3), dtype=np.uint8)
        sequence = np.concatenate([sequence, pad_array])
    elif len(sequence) > max_seq_length:
        sequence = sequence[:max_seq_length]
    return sequence

for person_id in people:
    tx1 = time.time()
    for data_type in data_types:
        for word_index, word in enumerate(folder):
            for iteration in instances:
                path = os.path.join(directory, person_id, data_type, word, iteration)
                filelist = sorted(os.listdir(path + '/'))
                sequence = []
                for img_name in filelist:
                    if img_name.startswith('color'):
                        image = imageio.imread(path + '/' + img_name)
                        image = resize(image, (MAX_WIDTH, MAX_HEIGHT))
                        image = 255 * image
                        image = image.astype(np.uint8)

                        # Simulate low-light conditions
                        dark_image = simulate_low_light(image, darkness_factor)

                        # Apply random flip
                        flipped_image = random_flip(dark_image)

                        sequence.append(flipped_image)

                # Pad or truncate the sequence to a uniform length
                if person_id in UNSEEN_TEST_SPLIT:
                    X_test.append(pad_or_truncate(sequence))
                    y_test.append(word_index)
                elif person_id in UNSEEN_VALIDATION_SPLIT:
                    X_val.append(pad_or_truncate(sequence))
                    y_val.append(word_index)
                else:
                    X_train.append(pad_or_truncate(sequence))
                    y_train.append(word_index)
    tx2 = time.time()
    print(f'Finished reading images for person {person_id}. Time taken : {tx2 - tx1} secs.')

X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

t2 = time.time()
print(f"Time taken for creating constant size 3D Tensors from those cropped lip regions: {t2 - t1} secs.")
